# A Mood Dashboard for the MAYke It! Hackathon

## Metrics
- Hours of Sleep per Night
- Minutes of Workout (Physical Activity)
- Time Spent Socializing in Minutes
- Energy Levels: On a scale from 1-10
- Mood: ten categories: "happy", "sad", "anxious", "depressed", "content", "angry", "excited", "bored", "stressed", "relaxed"
- Screen Time in Minutes
- Sleep Quality on a scale from 1-10
- Rainfall in Millimeters
- Sunshine in Hours

## Generating a synthetic dataset for these metrics

In [1]:
import random
import pandas as pd
from datetime import datetime, timedelta

In [2]:
moods = ["happy", "sad", "anxious", "depressed", "content", 
         "angry", "excited", "bored", "stressed", "relaxed"]

In [3]:
def generate_synthetic_data(num_days):
    data = []
    current_date = datetime.now()
    good_moods = ["happy", "content", "excited", "relaxed"]
    bad_moods = ["sad", "depressed", "bored", "anxious", "stressed", "angry"]
    mixed_moods = ["content", "relaxed", "bored"]

    for i in range(num_days):
        date = current_date - timedelta(days=i)
        mood = random.choice(good_moods + bad_moods)

        if mood in good_moods:
            sleep_hours = round(random.uniform(6, 9), 2)
            workout_minutes = random.randint(30, 120)
            social_interaction_minutes = random.randint(60, 300)
            energy_level = random.randint(7, 10)
        else:
            sleep_hours = round(random.uniform(4, 7), 2)
            workout_minutes = random.randint(0, 60)
            social_interaction_minutes = random.randint(0, 120)
            energy_level = random.randint(1, 6)

        if mood in mixed_moods:
            sleep_quality = random.randint(7, 10)
        else:
            sleep_quality = random.randint(1, 6)

        if mood in bad_moods:
            screen_time_minutes = random.randint(240, 480)
        else:
            screen_time_minutes = random.randint(60, 240)

        rainfall_mm = round(random.uniform(0, 100), 2)
        sunshine_hours = round(random.uniform(0, 12), 2)

        data.append((date, sleep_hours, workout_minutes, social_interaction_minutes, energy_level, mood, screen_time_minutes, sleep_quality, rainfall_mm, sunshine_hours))
    
    return pd.DataFrame(data, columns=['date', 'sleep_hours', 'workout_minutes', 'social_interaction_minutes', 'energy_level', 'mood', 'screen_time_minutes', 'sleep_quality', 'rainfall_mm', 'sunshine_hours'])

In [4]:
num_days = 30
synthetic_data = generate_synthetic_data(num_days)
print(synthetic_data.head())

                        date  sleep_hours  workout_minutes  \
0 2024-05-19 18:25:57.730583         4.14               46   
1 2024-05-18 18:25:57.730583         4.85               35   
2 2024-05-17 18:25:57.730583         8.38               39   
3 2024-05-16 18:25:57.730583         8.32               31   
4 2024-05-15 18:25:57.730583         8.16               69   

   social_interaction_minutes  energy_level     mood  screen_time_minutes  \
0                          51             1    bored                  316   
1                          37             1    angry                  289   
2                         276            10    happy                  157   
3                         110            10  excited                  224   
4                         177             9  content                  144   

   sleep_quality  rainfall_mm  sunshine_hours  
0              9        61.15            1.59  
1              2        21.07            1.09  
2              3    

In [5]:
synthetic_data.to_csv("data/synthetic_mood_data.csv", index=False)

In [6]:
#!pip install psycopg2
!pip install python-dotenv

## Building the dashboard

In [7]:
!pip install dash cloudpickle fsspec partd pyyaml toolz locket msgpack sortedcontainers tblib zict

In [8]:
#!pip uninstall -y typing_extensions sqlalchemy
!pip install typing_extensions==4.5.0 sqlalchemy==1.4.35

In [9]:
!pip install dash-bootstrap-components

In [16]:
import os
import psycopg2
from sqlalchemy import create_engine
from dotenv import load_dotenv
import pandas as pd
from datetime import datetime, timedelta
import random
import plotly.express as px
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc
import webbrowser
from sqlalchemy.sql import text

# Load environment variables from .env file
load_dotenv('postgres.env')

# Retrieve environment variables
db_user = os.getenv("PGUSER")
db_password = os.getenv("PGPASSWORD")
db_host = os.getenv("PGHOST")
db_name = os.getenv("PGDATABASE")
db_port = os.getenv("PGPORT")

# Create SQLAlchemy engine
engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')

def insert_data(df):
    conn_str = f"postgresql://{db_user}:{db_password}@{db_host}/{db_name}"
    engine = create_engine(conn_str)
    
    try:
        with engine.connect() as conn:
            # Check if a record with the specified date already exists
            query = "SELECT COUNT(*) FROM user_activities WHERE date = :date"
            for index, row in df.iterrows():
                result = conn.execute(text(query), {"date": row['date']})
                count = result.scalar()

                if count == 0:
                    # If no record exists, insert the data
                    row_df = pd.DataFrame([row])
                    row_df.to_sql('user_activities', conn, if_exists='append', index=False)
                    print(f"Data inserted successfully for date: {row['date']}")
                else:
                    print(f"Data for date {row['date']} already exists. Skipping insertion.")

    except Exception as e:
        print(f"Error inserting data: {e}")

def reset_data():
    conn_str = f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"
    engine = create_engine(conn_str)

    try:
        with engine.connect() as conn:
            conn.execute(text("DROP TABLE IF EXISTS user_activities"))
            print("Table dropped successfully.")
            # Recreate the table
            create_table_query = """
            CREATE TABLE user_activities (
                id SERIAL PRIMARY KEY,
                date DATE NOT NULL,
                sleep_hours FLOAT NOT NULL,
                workout_minutes INT NOT NULL,
                social_interaction_minutes INT NOT NULL,
                energy_level INT NOT NULL,
                mood VARCHAR(50) NOT NULL,
                screen_time_minutes INT NOT NULL,
                sleep_quality INT NOT NULL,
                rainfall_mm FLOAT NOT NULL,
                sunshine_hours FLOAT NOT NULL
            );
            """
            conn.execute(text(create_table_query))
            print("Table created successfully.")
    except Exception as e:
        print(f"Error dropping or creating table: {e}")

# if you want to drop the table, otherwise comment it out
reset_data()        
num_days=30
synthetic_data = generate_synthetic_data(num_days)
insert_data(synthetic_data)

def fetch_data():
    engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')
    query = "SELECT * FROM user_activities;"
    df = pd.read_sql(query, engine)
    return df

# Load data
df = fetch_data()

external_stylesheets = [dbc.themes.BOOTSTRAP]

# Initialize the Dash app
app = dash.Dash(__name__, external_stylesheets=external_stylesheets,  assets_folder='assets')

app.layout = html.Div([
    html.Iframe(
        src='https://arinakamazu.github.io/Pomodoro_maykit_project/',
        style={"width": "100%", "height": "600px", "border": "none"}
    ),
    html.Div([
        html.Iframe(
            src='https://open.spotify.com/embed/playlist/2HGrok5WAHWVIGDhATZdTG?utm_source=generator&theme=0', 
            style={"border-radius": "12px", "width": "100%", "height": "352px", "frameborder": "0", "allow": "autoplay; clipboard-write; encrypted-media; fullscreen; picture-in-picture", "loading": "lazy"}
        )
    ], style={'margin': 'auto', 'width': '60%', 'padding': '10px'}),
    html.H3("Daily Tasks", style={'text-align': 'center', 'margin-top': '20px', 'color': '#ffffff'}),
    html.Div([
        html.Iframe(
            src='https://todoist.com/app/embed/project', 
            style={"border-radius": "12px", "width": "100%", "height": "500px", "frameborder": "0", "allow": "autoplay; clipboard-write; encrypted-media; fullscreen; picture-in-picture", "loading": "lazy"})], style={'background-color': '#1c1c2e', 'margin': 'auto', 'width': '80%', 'padding': '10px'}),
    html.H3("Mood Analysis for the Motivated", style={'text-align': 'center', 'margin-top': '20px', 'color': '#ffffff'}),
    html.Div([
        html.Div(dcc.Graph(id='mood-bar-chart'), style={'flex': '1 1 calc(50% - 20px)', 'margin': '10px'}),
        html.Div(dcc.Graph(id='sleep-energy-scatter-plot'), style={'flex': '1 1 calc(50% - 20px)', 'margin': '10px'}),
    ], style={'display': 'flex', 'justify-content': 'center', 'flex-wrap': 'wrap'}),
    html.Div([
        html.Div(dcc.Graph(id='sleep-quality-mood-chart'), style={'flex': '1 1 calc(50% - 20px)', 'margin': '10px'}),
        html.Div(dcc.Graph(id='activity-screen-time-mood-chart'), style={'flex': '1 1 calc(50% - 20px)', 'margin': '10px'}),
    ], style={'display': 'flex', 'justify-content': 'center', 'flex-wrap': 'wrap'}),
    html.Div([
        html.Div(dcc.Graph(id='social-interaction-line-chart'), style={'flex': '1 1 calc(50% - 20px)', 'margin': '10px'}),
        html.Div(dcc.Graph(id='weather-mood-scatter-plot'), style={'flex': '1 1 calc(50% - 20px)', 'margin': '10px'}),
    ], style={'display': 'flex', 'justify-content': 'center', 'flex-wrap': 'wrap'}),
    html.Div(id='submission-status'),
    # Button to open the modal
    html.Div([
        dbc.Button("Enter Your Data", id="open-modal", n_clicks=0, style={'background-color': '#6a0dad', 'color': '#ffffff', 'margin': '10px'})
    ], style={'text-align': 'center'}),
    dbc.Modal([
        dbc.ModalHeader("Enter Your Data", style={'background-color': '#000000', 'color': '#ffffff', 'border-bottom': 'none'}),
        dbc.ModalBody([
            dbc.Row([
                dbc.Col(dcc.DatePickerSingle(id='date-picker', date=datetime.now().date()), width=6),
                dbc.Col(dcc.Input(id='sleep-hours', type='number', placeholder='Sleep Hours', min=0, max=24, step=0.1), width=6),
            ], style={'margin-bottom': '10px'}),
            dbc.Row([
                dbc.Col(dcc.Input(id='workout-minutes', type='number', placeholder='Workout Minutes', min=0, max=300), width=6),
                dbc.Col(dcc.Input(id='social-minutes', type='number', placeholder='Social Interaction Minutes', min=0, max=1440), width=6),
            ], style={'margin-bottom': '10px'}),
            dbc.Row([
                dbc.Col(dcc.Input(id='energy-level', type='number', placeholder='Energy Level (1-10)', min=1, max=10), width=6),
                dbc.Col(dcc.Dropdown(id='mood', options=[{'label': mood, 'value': mood} for mood in ["happy", "content", "excited", "relaxed", "sad", "depressed", "bored", "anxious", "stressed", "angry"]], placeholder='Select Mood'), width=6),
            ], style={'margin-bottom': '10px'}),
            dbc.Row([
                dbc.Col(dcc.Input(id='screen-time', type='number', placeholder='Screen Time Minutes', min=0, max=1440), width=6),
                dbc.Col(dcc.Input(id='sleep-quality', type='number', placeholder='Sleep Quality (1-10)', min=1, max=10), width=6),
            ], style={'margin-bottom': '10px'}),
            dbc.Row([
                dbc.Col(dcc.Input(id='rainfall', type='number', placeholder='Rainfall (mm)', min=0, step=0.1), width=6),
                dbc.Col(dcc.Input(id='sunshine', type='number', placeholder='Sunshine Hours', min=0, max=24, step=0.1), width=6),
            ], style={'margin-bottom': '10px'}),
        ], style={'background-color': '#000000', 'color': '#ffffff'}),
        dbc.ModalFooter([
            dbc.Button("Submit", id="submit-button", n_clicks=0, style={'background-color': '#6a0dad', 'color': '#ffffff', 'font-size': '16px', 'font-weight': 'bold', 'border': 'none'}),
            dbc.Button("Close", id="close-modal", n_clicks=0, style={'background-color': '#6a0dad', 'color': '#ffffff', 'font-size': '16px', 'font-weight': 'bold', 'border': 'none'})
        ], style={'background-color': '#000000', 'border-top': 'none'}),
    ], id='data-entry-modal', is_open=False, size='lg'),
])
app.index_string = '''
<!DOCTYPE html>
<html>
    <head>
        {%metas%}
        <title>{%title%}</title>
        {%favicon%}
        {%css%}
        <style>
            body {
                background-color: #1c1c2e; /* A dark color matching the timer image */
                color: #ffffff; /* White color for text to stand out against the dark background */
            }
            .dashboard-container {
                background-color: #1c1c2e;
                color: #ffffff;
            }
            .dashboard-header {
                background-color: #2b2b44; /* Slightly lighter but still dark */
                color: #ffffff;
                padding: 10px;
                border-radius: 10px;
                text-align: center;
            }
            .dashboard-section {
                background-color: #2b2b44;
                color: #ffffff;
                padding: 20px;
                border-radius: 10px;
                margin-bottom: 20px;
            }
            .dashboard-widget {
                background-color: #3a3a5b;
                color: #ffffff;
                padding: 10px;
                border-radius: 5px;
                margin-bottom: 10px;
            }
            iframe {
                border: none;
            }
            hr {
                border: none;
                border-top: 1px solid #1c1c2e; /* Change the border color to match the background */
            }
        </style>
    </head>
    <body>
        {%app_entry%}
        <footer>
            {%config%}
            {%scripts%}
            {%renderer%}
        </footer>
    </body>
</html>
'''

@app.callback(
    Output('data-entry-modal', 'is_open'),
    [Input('open-modal', 'n_clicks'), Input('close-modal', 'n_clicks')],
    [State('data-entry-modal', 'is_open')]
)
def toggle_modal(n1, n2, is_open):
    if n1 or n2:
        return not is_open
    return is_open

@app.callback(
    [Output('submit-button', 'children'), Output('submission-status', 'children'), 
     Output('date-picker', 'date'), Output('sleep-hours', 'value'), Output('workout-minutes', 'value'), 
     Output('social-minutes', 'value'), Output('energy-level', 'value'), Output('mood', 'value'), 
     Output('screen-time', 'value'), Output('sleep-quality', 'value'), Output('rainfall', 'value'), 
     Output('sunshine', 'value')],
    [Input('submit-button', 'n_clicks')],
    [State('date-picker', 'date'),
     State('sleep-hours', 'value'),
     State('workout-minutes', 'value'),
     State('social-minutes', 'value'),
     State('energy-level', 'value'),
     State('mood', 'value'),
     State('screen-time', 'value'),
     State('sleep-quality', 'value'),
     State('rainfall', 'value'),
     State('sunshine', 'value')]
)
def update_output(n_clicks, date, sleep_hours, workout_minutes, social_minutes, energy_level, mood, screen_time, sleep_quality, rainfall, sunshine):
    if n_clicks > 0:
        if not (date and sleep_hours is not None and workout_minutes is not None and social_minutes is not None and energy_level is not None and mood and screen_time is not None and sleep_quality is not None and rainfall is not None and sunshine is not None):
            return ('Submit', 'Please fill in all fields.', dash.no_update, dash.no_update, 
                    dash.no_update, dash.no_update, dash.no_update, dash.no_update, 
                    dash.no_update, dash.no_update, dash.no_update, dash.no_update)
        
        if mood == 'Select Mood':
            return ('Submit', 'Please select a valid mood.', dash.no_update, dash.no_update, 
                    dash.no_update, dash.no_update, dash.no_update, dash.no_update, 
                    dash.no_update, dash.no_update, dash.no_update, dash.no_update)

        new_data = pd.DataFrame([{
            'date': date,
            'sleep_hours': sleep_hours,
            'workout_minutes': workout_minutes,
            'social_interaction_minutes': social_minutes,
            'energy_level': energy_level,
            'mood': mood,
            'screen_time_minutes': screen_time,
            'sleep_quality': sleep_quality,
            'rainfall_mm': rainfall,
            'sunshine_hours': sunshine
        }])
        
        insert_data(new_data)
        global df
        df = fetch_data()
        return ('Submit', 'Data submitted successfully!', None, None, None, None, 
                None, None, None, None, None, None)
    return ('Submit', '', dash.no_update, dash.no_update, dash.no_update, dash.no_update, 
            dash.no_update, dash.no_update, dash.no_update, dash.no_update, dash.no_update, dash.no_update)

@app.callback(
    Output('mood-bar-chart', 'figure'),
    Input('mood-bar-chart', 'id')
)
def update_mood_bar_chart(_):
    # Aggregate mood counts
    mood_counts = df['mood'].value_counts().reset_index()
    mood_counts.columns = ['mood', 'count']
    fig = px.bar(mood_counts, x='mood', y='count', title='Mood Counts Over Last 30 Days', 
                 color_discrete_sequence=['#6a0dad']) #'#6a0dad' '#ff6f61'
    fig.update_layout(
        plot_bgcolor='#1c1c2e', 
        paper_bgcolor='#1c1c2e', 
        font_color='#ffffff', 
        title_font_color='#ffffff', 
        xaxis=dict(color='#ffffff'), 
        yaxis=dict(color='#ffffff'),
        width=600, 
        height=400
    )
    return fig

@app.callback(
    Output('sleep-energy-scatter-plot', 'figure'),
    Input('sleep-energy-scatter-plot', 'id')
)
def update_sleep_energy_scatter_plot(_):
    fig = px.scatter(df, x='sleep_hours', y='energy_level', trendline='ols', title='Sleep Hours vs. Energy Level', 
                     color_discrete_sequence=['#32CD32'])
    fig.update_layout(
        plot_bgcolor='#1c1c2e', 
        paper_bgcolor='#1c1c2e', 
        font_color='#ffffff', 
        title_font_color='#ffffff', 
        xaxis=dict(color='#ffffff'), 
        yaxis=dict(color='#ffffff'),
        width=600, 
        height=400
    )
    return fig

@app.callback(
    Output('sleep-quality-mood-chart', 'figure'),
    Input('sleep-quality-mood-chart', 'id')
)
def update_sleep_quality_mood_chart(_):
    fig = px.box(df, x='mood', y='sleep_quality', title='Sleep Quality vs. Mood', 
                 color_discrete_sequence=['#FF1493'])
    fig.update_layout(
        plot_bgcolor='#1c1c2e', 
        paper_bgcolor='#1c1c2e', 
        font_color='#ffffff', 
        title_font_color='#ffffff', 
        xaxis=dict(color='#ffffff'), 
        yaxis=dict(color='#ffffff'),
        width=600, 
        height=400
    )
    return fig

@app.callback(
    Output('activity-screen-time-mood-chart', 'figure'),
    Input('activity-screen-time-mood-chart', 'id')
)
def update_activity_screen_time_mood_chart(_):
    # Aggregate workout minutes and screen time minutes by mood
    agg_activity = df.groupby('mood', as_index=False)['workout_minutes'].mean(numeric_only=True)
    agg_screen_time = df.groupby('mood', as_index=False)['screen_time_minutes'].mean(numeric_only=True)

    # Combine the data
    agg_combined = pd.merge(agg_activity, agg_screen_time, on='mood')
    fig = px.line(agg_combined, x='mood', y=['workout_minutes', 'screen_time_minutes'], title='Average Physical Activity and Screen Time vs. Mood')
    fig.update_traces(line=dict(color='#FF1493'), selector=dict(name='workout_minutes'))
    fig.update_traces(line=dict(color='#00CED1'), selector=dict(name='screen_time_minutes'))  # Changed color to cyan for better visibility
    fig.update_layout(
        plot_bgcolor='#1c1c2e', 
        paper_bgcolor='#1c1c2e', 
        font_color='#ffffff', 
        title_font_color='#ffffff', 
        xaxis=dict(color='#ffffff'), 
        yaxis=dict(color='#ffffff'),
        width=600, 
        height=400
    )
    return fig

@app.callback(
    Output('social-interaction-line-chart', 'figure'),
    Input('social-interaction-line-chart', 'id')
)
def update_social_interaction_line_chart(_):
    df_grouped = df.groupby('date').mean(numeric_only=True).reset_index()
    fig = px.line(df_grouped, x='date', y='social_interaction_minutes', title='Average Social Interaction Minutes Over Last 30 Days', 
                  color_discrete_sequence=['#FFD700'])
    fig.update_layout(
        plot_bgcolor='#1c1c2e', 
        paper_bgcolor='#1c1c2e', 
        font_color='#ffffff', 
        title_font_color='#ffffff', 
        xaxis=dict(color='#ffffff'), 
        yaxis=dict(color='#ffffff'),
        width=600, 
        height=400
    )
    return fig

@app.callback(
    Output('weather-mood-scatter-plot', 'figure'),
    Input('weather-mood-scatter-plot', 'id')
)
def update_weather_mood_scatter_plot(_):
    fig = px.scatter(df, x='rainfall_mm', y='sunshine_hours', color='mood', title='Effects of Rainfall and Sunshine Hours on Mood', 
    color_discrete_map = {
        'happy': '#FFD700',       # Golden Yellow
        'content': '#FF4500',     # Sunset Orange
        'excited': '#FF1493',     # Vibrant Pink
        'relaxed': '#4B0082',     # Deep Purple
        'sad': '#3366CC',         # Blue
        'depressed': '#3399FF',   # Light Blue
        'bored': '#66CCFF',       # Sky Blue
        'anxious': '#00CED1',     # Dark Turquoise
        'stressed': '#32CD32',    # Lime Green
        'angry': '#DC143C'        # Crimson
    }
)
    fig.update_layout(
        plot_bgcolor='#1c1c2e', 
        paper_bgcolor='#1c1c2e', 
        font_color='#ffffff', 
        title_font_color='#ffffff', 
        xaxis=dict(color='#ffffff'), 
        yaxis=dict(color='#ffffff'),
        width=600, 
        height=400
    )
    return fig

# Run the app on a different port and open in a new tab
if __name__ == '__main__':
    port = 8055
    webbrowser.open_new_tab(f"http://127.0.0.1:{port}")
    app.run(debug=True, port=port)

Table dropped successfully.
Table created successfully.
Data inserted successfully for date: 2024-05-19 18:41:24.017721
Data inserted successfully for date: 2024-05-18 18:41:24.017721
Data inserted successfully for date: 2024-05-17 18:41:24.017721
Data inserted successfully for date: 2024-05-16 18:41:24.017721
Data inserted successfully for date: 2024-05-15 18:41:24.017721
Data inserted successfully for date: 2024-05-14 18:41:24.017721
Data inserted successfully for date: 2024-05-13 18:41:24.017721
Data inserted successfully for date: 2024-05-12 18:41:24.017721
Data inserted successfully for date: 2024-05-11 18:41:24.017721
Data inserted successfully for date: 2024-05-10 18:41:24.017721
Data inserted successfully for date: 2024-05-09 18:41:24.017721
Data inserted successfully for date: 2024-05-08 18:41:24.017721
Data inserted successfully for date: 2024-05-07 18:41:24.017721
Data inserted successfully for date: 2024-05-06 18:41:24.017721
Data inserted successfully for date: 2024-05-05 